In [2]:
import os
import librosa
import gc
import ast
import numpy as np
import pandas as pd
from pathlib import Path
# from tqdm import tqdm
import torchaudio  # torchaudio==2.9.0  torch==2.9.0 torchcodec==0.8
import sys
import datasets  # pip install datasets==3.6.0
from datasets import load_dataset
import evaluate
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

In [3]:
import sys
import torch
from torch.utils.data import DataLoader, Subset
from dotenv import load_dotenv
import os
from pathlib import Path

load_dotenv() 

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load paths from environment variables
data_root = Path(os.getenv("DATA_ROOT", "./data"))  # default fallback to ./data
image_data_path = "audio/for-norm/for-norm/"
data_root = data_root/ image_data_path

print(f"Data directory: {data_root}")
print(f"Exists: {data_root.exists()}")

# Add src to path
src_path = Path(r"D:\florida_coursework\third_sem\multimedia_expert_systems\multimedia_prototype\mutlimedia_mvp\src")
sys.path.insert(0, str(src_path))



Using device: cuda
Data directory: D:\florida_coursework\third_sem\multimedia_expert_systems\multimedia_prototype\data\audio\for-norm\for-norm
Exists: True


In [4]:
full_dataset = load_dataset(
    "audiofolder",
    data_dir=data_root,
)

Resolving data files:   0%|          | 0/53868 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/10798 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/4634 [00:00<?, ?it/s]

In [5]:
full_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 53868
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 10798
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 4634
    })
})

In [6]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

model_name = "facebook/wav2vec2-base"

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
encoder = Wav2Vec2Model.from_pretrained(model_name)

C:\Users\muthu\miniconda3\envs\multi_env\Lib\site-packages\transformers\configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [7]:
max_duration = 5.0  # seconds (you can change this)

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]

    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        padding="max_length",
        return_attention_mask=True,
        return_tensors="pt"
    )
    
    inputs["labels"] = examples["label"]
    return inputs



In [8]:
from datasets import DatasetDict
test_only = DatasetDict({
    "test": full_dataset["test"]
})


In [9]:
processed_dataset = test_only.map(
    preprocess_function,
    batched=True,
    remove_columns=["audio","label"],
)

Map:   0%|          | 0/4634 [00:00<?, ? examples/s]

In [10]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

test_loader = DataLoader(
    processed_dataset["test"],
    batch_size=8,
    shuffle=True,
    collate_fn=default_data_collator  # <-- important!
)

# test_loader = DataLoader(
#     processed_dataset["test"],
#     batch_size=8,
#     collate_fn=default_data_collator
# )

In [11]:
import torch
import torch.nn as nn

class AudioEmbeddingModel(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        if hasattr(self.encoder, "freeze_feature_extractor"):
            self.encoder.freeze_feature_extractor()

    def forward(self, input_values, attention_mask):
        outputs = self.encoder(
            input_values=input_values,
            attention_mask=attention_mask
        )
        
        hidden_states = outputs.last_hidden_state  # (B, T_hidden, H)
        
        # HuggingFace Wav2Vec2 / WavLM already applies attention masking internally
        # So you can just do mean pooling over time dimension
        embedding = hidden_states.mean(dim=1)     # (B, H)
        
        return embedding


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = AudioEmbeddingModel(encoder).to(device)
embedding_model.eval()


C:\Users\muthu\miniconda3\envs\multi_env\Lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1367: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


AudioEmbeddingModel(
  (encoder): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (enco

In [13]:
all_audio_embs = []
all_labels = []

embedding_model.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        print(f"  Processing batch {i+1}/{len(test_loader)}...")
        input_values = batch["input_values"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"]  # still available as tensor

        emb = embedding_model(input_values, attention_mask)
        all_audio_embs.append(emb.cpu())
        all_labels.append(labels)  # collect labels


  Processing batch 1/580...
  Processing batch 2/580...
  Processing batch 3/580...
  Processing batch 4/580...
  Processing batch 5/580...
  Processing batch 6/580...
  Processing batch 7/580...
  Processing batch 8/580...
  Processing batch 9/580...
  Processing batch 10/580...
  Processing batch 11/580...
  Processing batch 12/580...
  Processing batch 13/580...
  Processing batch 14/580...
  Processing batch 15/580...
  Processing batch 16/580...
  Processing batch 17/580...
  Processing batch 18/580...
  Processing batch 19/580...
  Processing batch 20/580...
  Processing batch 21/580...
  Processing batch 22/580...
  Processing batch 23/580...
  Processing batch 24/580...
  Processing batch 25/580...
  Processing batch 26/580...
  Processing batch 27/580...
  Processing batch 28/580...
  Processing batch 29/580...
  Processing batch 30/580...
  Processing batch 31/580...
  Processing batch 32/580...
  Processing batch 33/580...
  Processing batch 34/580...
  Processing batch 35/5

In [24]:
all_audio_embs = torch.cat(all_audio_embs, dim=0)   # (N, H)
all_labels = torch.cat(all_labels, dim=0)           # (N,)

In [25]:
save_path = "embeddings/test_audio_embeddings.pt"
torch.save({
    "embeddings": all_audio_embs,   # (N, D)
    "labels": all_labels            # (N,)
}, save_path)

print("Saved:", save_path)

Saved: embeddings/test_audio_embeddings.pt
